In [1]:
import ase

In [2]:
import keras
from keras.models import load_model

Using TensorFlow backend.


# Module: Numerical Force Calcalation #
This module is designed the develop the necessary for the calculating the numerical derivative of the potential energy surface. 

## Mode 1: Load All the Subnets ##

In [3]:
import sys
sys.path.insert(0,"../") # Add the directory to Python Path to import functions from other modules for testing. 
print(sys.path)

['../', '', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/root/.local/lib/python3.5/site-packages', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.5/dist-packages/IPython/extensions', '/root/.ipython']


In [4]:
import pandas as pd

from model_io import read_np_arr_from_pickle
from model_io import read_keras_subnet_ind

In [5]:
h_subnet = read_keras_subnet_ind("H-subnet")
o_subnet = read_keras_subnet_ind("O-subnet")
n_subnet = read_keras_subnet_ind("N-subnet")
c_subnet = read_keras_subnet_ind("C-subnet")
rc_subnet = read_keras_subnet_ind('rc-subnet')

In [6]:
subnet_arr = pd.Series([])
subnet_arr['H'] = h_subnet
subnet_arr['O'] = o_subnet
subnet_arr['C'] = c_subnet
subnet_arr['N'] = n_subnet
subnet_arr['rc'] = rc_subnet

## Mode 2: Load Entire Model ##

In [7]:
model = load_model("../YeltsinNet/model.h5")

## 1.0 Change the Numerical Atom Position ##

In [8]:
from ForceNum import shake_atom_posit
from ForceNum import shake_mol_posit

In [9]:
from ase.build import molecule
atoms_test = molecule('H2O') 
new_atom_posit = shake_atom_posit(atoms_test[0], 'x', 0.001)

posit_arr = shake_mol_posit(atoms_test, 0, 0.001)
posit_arr.shape
print(posit_arr)

[[0.001    0.       0.119262]
 [0.       0.001    0.119262]
 [0.       0.       0.120262]]


In [10]:
def num_deriv_3point():
    """Calcalates the numerical derivative based on the 3 point method. 
     
    """

# Build the ASE Calculator Model #

In [11]:
import os
import numpy as np
from ase.calculators.calculator import Calculator, all_changes

# TODO: Implement the vectorization of individual atoms in the vectorization module.
def atoms_to_input(atoms):
    inp_atom = []
    
    
    
    pass



In [12]:
def symm_func_ind(at_idx_map, Gparam_dict, atoms, atom_idx):
    """ !!!! Serve Only as a Template for symm_func_ind
    calculate symmetry functions derivative. 
    For this function, I just want one of the coordinates. Not an array of everything
    
        Args:
            distances: distance values; pandas dataframe of shape (nb_samples, nb_distances)
            at_idx_map: a mapping between atom types and atom indexes; dictionary
            Gparam_dict: symmetry function parameters;
                            dictionary with 1st layer keys  = atom types,
                                2nd layer keys = symmetry function types (radial or angular)
                                values = 2D arrays of sym. function parameters of
                                shape (nb_sym_functions, nb_filter_parameters)
            atom: ase.atoms object that represent that particular atom 

        Outputs: 
           Gfunc_data: symmetry function values;
                        dictionary with 1st layer keys = atom types,
                            2nd layer keys = atom indexes,
                            values = 2D arrays with shape=(nb_samples, nb_sym_functions)
                            
    Comment:
    This function is temporary. More Radical changes are expected. 
    Just help to develop the framework. 
    """


    # This for loop goes through elements
    # Are together
    at_type = atoms[atom_idx].symbol
    Gparam_rad = Gparam_dict[at_type]['rad'] # Problem Here
    Gparam_ang = Gparam_dict[at_type]['ang'] # Problem Here

    rad_count = sum([Gparam_rad[t].shape[0] for t in Gparam_rad.keys()])
    ang_count = sum([Gparam_ang[t].shape[0] for t in Gparam_ang.keys()])
    xyz_i = atoms[atom_idx].position


    ## This for loop goes through all the atoms (belong to the same element)
    Gfunc_data = np.zeros(((rad_count + ang_count), 3))

    G_temp_count = 0


    # radial components
    # Loop through different elements
    for at2_type in Gparam_rad.keys():
        comp_count =  Gparam_rad[at2_type].shape[0]
        G_temp_component = np.zeros((comp_count, 3)) # Will store the array of the [dx, dy, dz]
        
        # One count is for one set of parameter values
        for count, values in enumerate(Gparam_rad[at2_type]):
            # Summing up the contribution from different distance values. 
            for at2 in at_idx_map[at2_type][at_idx_map[at2_type]!=atom_idx]:
                R12 = distance_ind(atoms[atom_idx], atoms[at2])
                
                xyz_j = atoms[at2].position
                # values[0] = Rs, values[1] = eta (integer, not array)
                # Then Calculate the radial symmetric function -> value of G.
                rad_temp = ddR_radial_filter(values[0], values[1], R12, xyz_i, xyz_j)
                G_temp_component[count, 0:3] += rad_temp

        Gfunc_data[G_temp_count:G_temp_count+comp_count, 0:3] = G_temp_component
        G_temp_count += comp_count

    # ======================
    # angular components
    for atAatB_type in Gparam_ang.keys():
        comp_count = Gparam_ang[atAatB_type].shape[0]
        G_temp_component = np.zeros((comp_count,3))

        # This for loop goes through all 'HH', 'HO' combo?
        for count, values in enumerate(Gparam_ang[atAatB_type]):
            atA_list = at_idx_map[atAatB_type[0]][at_idx_map[atAatB_type[0]]!=atom_idx]
            for atA in atA_list:

                R1A = distance_ind(atoms[atom_idx], atoms[atA]);
                
                if atAatB_type[0] == atAatB_type[1]:
                    atB_list = at_idx_map[atAatB_type[1]][(at_idx_map[atAatB_type[1]]!=atom_idx) & (at_idx_map[atAatB_type[1]]>atA)]
                else:
                    atB_list = at_idx_map[atAatB_type[1]][(at_idx_map[atAatB_type[1]]!=atom_idx)]

                for atB in atB_list:
                    #R1B = 1
                    #RAB = 1
                    
                    R1B = distance_ind(atoms[atom_idx], atoms[atB])
                    RAB = distance_ind(atoms[atA], atoms[atB])               
                    
                    xyz_j = atoms[atA].position
                    xyz_k = atoms[atB].position
                                   

                    ang_temp = ddR_angular_filter(R1A, R1B, RAB, values[0], values[1], values[2], xyz_i, xyz_j, xyz_k)
                    

                    G_temp_component[ count, 0:3] += ang_temp
                    
                    # For Debugging
                    
        Gfunc_data[G_temp_count:G_temp_count+comp_count, 0:3] = G_temp_component
        G_temp_count += comp_count
    return Gfunc_data    

## 2.0 Get the energy ##

## 3.0 Using the 3-point method to calculate the derivative ##